In [19]:
import sys
sys.path.append('..')
import pandas as pd

In [20]:
from settings import DBSelector

In [21]:
db = DBSelector()

In [22]:
engine = db.get_engine('db_stock','qq')

In [7]:
table = 'bond'
df = pd.read_sql(table,con=engine)

In [16]:
df.head()

,可转债代码,可转债名称,可转债价格,正股名称,正股代码,正股现价,正股涨跌幅,最新转股价,溢价率,可转债涨幅,回售触发价,转股起始日,到期时间,成交额(万元),强赎价格,剩余时间,回售起始日,评级,下修提示,下调次数,转债剩余占总市值比,剩余规模,发行规模,股东配售率,强赎标志,强赎日期,更新日期
0,113581,龙蟠转债,257.300,龙蟠科技,603906,27.61,-2.44,9.48,-11.65,-5.18,6.64,2020-10-29,26-04-22,37567.25,120.0,5.622,2024-04-22,A+,,0,4.8,4.000,4.000,76.13,X,None,2020-09-08 15:03
1,123063,大禹转债,130.502,大禹节水,300021,7.05,12.44,4.94,-8.55,3.48,3.46,2021-02-03,26-07-27,48202.03,120.0,5.885,2025-07-28,AA-,,0,11.3,6.380,6.380,68.99,X,None,2020-09-08 15:03
2,113575,东时转债,143.020,东方时尚,603377,22.76,-1.39,14.56,-8.51,-1.81,10.19,2020-10-15,26-04-08,1368.22,108.0,5.584,2024-04-08,AA,,0,3.2,4.280,4.280,20.73,X,None,2020-09-08 15:03
3,123050,聚飞转债,145.000,聚飞光电,300303,8.15,3.43,5.18,-7.84,-0.92,3.63,2020-10-20,26-04-13,31066.36,118.0,5.597,2024-04-15,AA-,,0,6.8,7.050,7.050,43.22,X,None,2020-09-08 15:03
4,123051,今天转债,138.282,今天国际,300532,13.02,3.42,8.80,-6.53,1.16,6.16,2020-12-10,26-06-03,7182.65,116.0,5.737,2024-06-03,A+,,0,7.9,2.800,2.800,76.80,X,None,2020-09-08 15:03


In [11]:
cyb_df = df[df['正股代码'].str.startswith('3')]

In [12]:
cyb_df['正股涨跌幅'].mean()

2.9328000000000003

In [13]:
non_cyb_df = df[~df['正股代码'].str.startswith('3')]

In [14]:
non_cyb_df['正股涨跌幅'].mean()

0.39790983606557384

In [17]:
cyb_df['可转债涨幅'].mean()

0.3807843137254903

In [18]:
non_cyb_df['可转债涨幅'].mean()

-0.2745491803278688

# 港股中签率分析

In [37]:
import pandas as pd
import numpy as np
table = 'tb_hk_new_stock'
df = pd.read_sql(table,con=engine)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 15 columns):
id                  497 non-null int64
name                497 non-null object
code                497 non-null object
issue_date          0 non-null datetime64[ns]
each_hand_stock     497 non-null object
share_value_Yi      497 non-null object
margin_price        497 non-null object
price               457 non-null float64
over_price_part     497 non-null object
hit_least_num       457 non-null float64
hit_ratio           457 non-null float64
current_price       493 non-null float64
first_day_raise     497 non-null float64
accumulate_raise    490 non-null float64
crawltime           0 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(6), int64(1), object(6)
memory usage: 58.3+ KB


In [39]:
df.head()

,id,name,code,issue_date,each_hand_stock,share_value_Yi,margin_price,price,over_price_part,hit_least_num,hit_ratio,current_price,first_day_raise,accumulate_raise,crawltime
0,233,洛钼瑞银八零七购(@EC),09987.HK,NaT,50,"1,961.42",468.00,412.0,51.3,14.0,25.0,NaN,0.0,NaN,NaT
1,234,农夫山泉,09633.HK,NaT,200,"970.40-1,069.93",19.50-21.50,21.0,"1,147.3",150.0,12.0,33.0,54.0,54.0,NaT
2,235,智勤控股,09913.HK,NaT,"4,000",5.20-6.40,0.52-0.64,1.0,18.2,20.0,80.0,0.0,-18.0,-23.0,NaT
3,236,泰格医药,03347.HK,NaT,100,94.22-107.07,88.00-100.00,100.0,413.4,60.0,5.0,110.0,13.0,10.0,NaT
4,237,立德教育,01449.HK,NaT,"1,000",14.00-17.33,2.10-2.60,2.0,26.9,10.0,20.0,1.0,-12.0,-42.0,NaT


## 获取第一天收盘大于0的新股数

In [42]:
win=df[df['first_day_raise']>0]

## 获取第一天收盘小于0的新股数

In [43]:
lost=df[df['first_day_raise']<0]

In [44]:
len(win)

254

In [45]:
len(lost)

181

In [48]:
len(win)/len(lost)*100

140.3314917127072

## 计算收益期望  概率x收益

##### 数据清洗，格式转换， 把 3,000 的字符转为 3000 int

In [53]:
def convertor(src):
    try:
        dst = int(src.strip().replace(',',''))
    except:
        dst = None
    return dst

In [54]:
df['each_hand_stock']=df['each_hand_stock'].map(convertor)

### 首日涨幅*中签率*每手股数*上市价格

In [57]:
E=df['first_day_raise']*df['hit_ratio']*df['each_hand_stock']*df['price']/100/100

In [59]:
E.sum()

43882.78999999999

In [61]:
len(E)

497

In [62]:
490*50

24500